# Plotly Interactive Plots

Some interactive plots to visualize the dataset with more details.

In [1]:
# imports

%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from tqdm import tqdm
tqdm.pandas()

In [2]:
# Test API function before running larger simulations
from examples.sim_trace import generate_trace_api

input_data = {
    "arrival_rate": 0.9, 
    "warm_service_rate": 1/2.016, 
    "cold_service_rate": 1/2.163,
    "expiration_threshold": 600, 
    "max_time": 10000,
}

generate_trace_api(input_data)

{'reqs_cold': 22,
 'reqs_total': 8991,
 'reqs_warm': 8969,
 'prob_cold': 0.0024468913357802245,
 'reqs_reject': 0,
 'prob_reject': 0.0,
 'lifespan_avg': 4040.078897215589,
 'inst_count_avg': 8.035784234499953,
 'inst_running_count_avg': 1.821932885445238,
 'inst_idle_count_avg': 6.213851349054714,
 'arrival_rate': 0.9,
 'warm_service_rate': 0.49603174603174605,
 'cold_service_rate': 0.46232085067036527,
 'expiration_threshold': 600,
 'max_time': 10000}

In [3]:
# Plot characteristics for different arrival rates and expiration thresholds
num_arrival_rates = 10
exp_thresholds = [60, 600, 1200, 1800]
exp_threshold_labels = ["1 min", "10 min", "20 min", "30 min"]
input_data['arrival_rate'] = list(np.repeat(np.logspace(-3,1,num_arrival_rates), len(exp_thresholds)))
input_data['expiration_threshold'] = exp_thresholds * num_arrival_rates
input_data['expiration_threshold_labels'] = exp_threshold_labels * num_arrival_rates

df = pd.DataFrame(data=input_data).reset_index(drop=True)

def generate_trace_apply(x):
    return pd.Series(generate_trace_api(x.to_dict()))

df = df.progress_apply(generate_trace_apply, axis=1)
df.tail()

100%|██████████| 40/40 [00:49<00:00,  1.23s/it]


,reqs_cold,reqs_total,reqs_warm,prob_cold,reqs_reject,prob_reject,lifespan_avg,inst_count_avg,inst_running_count_avg,inst_idle_count_avg,arrival_rate,warm_service_rate,cold_service_rate,expiration_threshold,max_time,expiration_threshold_labels
35,23,36096,36073,0.000637,0,0.0,6958.846170,19.381820,7.300904,12.080916,3.593814,0.496032,0.462321,1800,10000,30 min
36,450,99871,99421,0.004506,0,0.0,769.061310,33.383044,20.214142,13.168902,10.000000,0.496032,0.462321,60,10000,1 min
37,58,100103,100045,0.000579,0,0.0,5376.081109,37.186850,20.147280,17.039570,10.000000,0.496032,0.462321,600,10000,10 min
38,50,99722,99672,0.000501,0,0.0,5608.465380,37.828662,20.165372,17.663290,10.000000,0.496032,0.462321,1200,10000,20 min
39,42,100702,100660,0.000417,0,0.0,7083.933928,38.721996,20.157138,18.564858,10.000000,0.496032,0.462321,1800,10000,30 min


In [4]:
# calculate utilizations
df['utilization_percent'] = df['inst_running_count_avg'] / df['inst_count_avg'] * 100
df['prob_cold_percent'] = df['prob_cold'] * 100

## Interactive Plots

We will be making interactive plots here using the data generated by our simulator.

In [9]:
# simple interactive plot
from plotly.subplots import make_subplots
import plotly.graph_objs as go

# create a dataframe for each expiration threshold
dfs = [df.loc[df['expiration_threshold_labels'] == name, :] for name in exp_threshold_labels]

# create traces
def df_add_trace(col_name, col_label, fig, axis_titles):
    for idx in range(len(exp_threshold_labels)):
        fig.add_trace(
            go.Scatter(
                x = dfs[idx].loc[:,'arrival_rate'],
                y = dfs[idx].loc[:,col_name],
                mode = 'markers+lines',
                name = exp_threshold_labels[idx],
            ),
        )

        axis_postfix = ""
        axis_titles["xaxis" + axis_postfix] = dict(title="Arrival Rate (reqs/sec)")
        axis_titles["yaxis" + axis_postfix] = dict(title=col_label)

    return axis_titles


def plot_fig_for_col(col_name, col_label):
    fig = go.Figure()
    axis_titles = {}
    axis_titles = df_add_trace(col_name, col_label, fig, axis_titles)
    fig.update_layout(
        xaxis_type="log",
        **axis_titles
        )
    fig.show()

In [10]:
plot_fig_for_col('prob_cold_percent', 'Prob of Cold Start (%)')

In [11]:
plot_fig_for_col('utilization_percent', 'Utilization (%)')

In [12]:
plot_fig_for_col('inst_idle_count_avg', 'Average Idle Instances')